<a href="https://colab.research.google.com/github/B0BWAX/MT5-TRANSLATION-FINETUNING/blob/main/MT5_TRANSLATION_SHOWCASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# loading tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
# loading model
import torch
model = torch.load('/content/drive/MyDrive/mt5_finetuned_model.pt')

In [4]:
# defining token mappings
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'fr': '<fr>',
    'es': '<es>',
    'ar': '<ar>'
}

In [5]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250104, 512)

In [6]:
# function to encode input
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]

In [7]:
def translate(input, target_lang):
    input_ids = encode_input_str(text = input, target_lang = output_language, tokenizer = tokenizer,
                                 seq_len = 128, lang_token_map = LANG_TOKEN_MAPPING)
    input_ids = input_ids.unsqueeze(0).cuda()

    translated_ids = model.generate(input_ids=input_ids, num_beams=10, length_penalty=0.2)

    translated_sentence = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_sentence

In [16]:
#@title Showcase
input_text = 'Je travaille sur ce projet depuis 5 jours.' #@param {type:"string"}
output_language = 'ar' #@param ["en", "ar", "fr", "es"]
print(translate(input_text, output_language))

انا تعمل على هذا المشروع منذ 5 يوما.
